####Lab 3.2 



Read in the reverse coding table and use it to make a dictionary using a dict comprehension.  The keys should be the question/column names and the values should be the same, but with the type added as a suffix.  HINT.  You should zip up the relevant columns.

Read in the health survey data and rename the columns using the dictionary from the previous step.

Write an expression that recodes each type of coding.  Do this using two column selectors, one for each type.

Next, we need to aggregate the values for each question type (e.g., questions starting with F1).  Do this using pl.reduce for each type.  Drop the original question columns using a column selector.

There are a couple of things we need to fix.  Copy your solution to the last part into a new cell and perform the following. 

Did you notice the null entries in the results from the last step.  This is due to nulls in the original data: polars treats the sum of values containing a null as null.  We can fix this be assigning a default value to each of the recoding from step 4. Adding this default will lead to a warning, follow the advise in the warning to remove it.

Instead of one expression for each questions type, use a list comprehension to combine them into one expression.  HINT. You will want to use .alias to assign the new column names.

In [212]:
import polars as pl
import operator 
from operator import add

In [213]:
healthcare = pl.read_csv("data/health_survey.csv")
healthcare.shape     
healthcare.head()

ID,F1,F5,F2,F1.1,F2.1,F6,F4,F3,F5.1,F1.2,F2.2,F6.1,F2.3,F4.1,F2.4,F5.2,F2.5,F6.2,F1.3,F2.6,F5.3,F4.2,F2.7,F3.1,F2.8,F5.4,F3.2,F1.4,F3.3,F1.5,F5.5,F6.3,F1.6,F5.6,F2.9,F3.4,F4.3,F2.10,F1.7,F6.4,F4.4,F5.7,F3.5,F2.11
i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
1,"""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree"""
2,"""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Neither Agree nor Disagree""","""Neither Agree nor Disagree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree"""
3,"""Strongly Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Strongly Agree""","""Strongly Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Strongly Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Disagree""","""Strongly Agree""","""Somewhat Agree""","""Strongly Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""",null,"""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Strongly Agree""","""Somewhat Disagree""","""Somewhat Disagree""","""Strongly Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Strongly Agree""","""Strongly Disagree""","""Somewhat Agree"""
4,"""Somewhat Agree""","""Somewhat Agree""","""Strongly Agree""","""Somewhat Agree""","""Strongly Agree""","""Neither Agree nor Disagree""","""Neither Agree nor Disagree""","""Somewhat Disagree""","""Somewhat Agree""","""Strongly Agree""","""Strongly Agree""","""Somewhat Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Strongly Agree""","""Somewhat Disagree""","""Strongly Agree""","""Somewhat Agree""","""Somewhat Agree""","""Strongly Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewh

In [214]:
reverse_coding = pl.read_csv("data/ReverseCodingItems.csv")
reverse_coding.head()

Question,Construct,Question # on Qualtrics Survey,Needs Reverse Coding?,Column Name
str,i64,i64,str,str
"""In the future, I plan to parti…",1,1,"""No""","""F1"""
"""Individuals are responsible fo…",5,2,"""Yes""","""F5"""
"""When tryng to understand the p…",2,3,"""No""","""F2"""
"""I plan to become involved in m…",1,4,"""No""","""F1.1"""
"""I can communicate well with ot…",2,5,"""No""","""F2.1"""


In [215]:
reverse_coding.filter(pl.col("Needs Reverse Coding?") == "Yes")

Question,Construct,Question # on Qualtrics Survey,Needs Reverse Coding?,Column Name
str,i64,i64,str,str
"""Individuals are responsible fo…",5,2,"""Yes""","""F5"""
"""It is hard for a group to func…",6,6,"""Yes""","""F6"""
"""I am a better follower than a …",4,14,"""Yes""","""F4.1"""
"""We need to look no further tha…",5,16,"""Yes""","""F5.2"""
"""People are poor because they c…",5,31,"""Yes""","""F5.5"""
"""I find it difficult to relate …",6,32,"""Yes""","""F6.3"""
"""I don't understand why some pe…",5,34,"""Yes""","""F5.6"""
"""I would rather have sombody el…",4,37,"""Yes""","""F4.3"""
"""I prefer the company of people…",6,40,"""Yes""","""F6.4"""


In [216]:
#2
rename_dict = {
    q: f"{q}_{'reverse' if need == 'Yes' else 'regular'}"
    for q, need in zip(
        reverse_coding["Column Name"].to_list(),
        reverse_coding["Needs Reverse Coding?"].to_list()
    )
}

healthcare = (healthcare.rename(rename_dict))
healthcare.head()

ID,F1_regular,F5_reverse,F2_regular,F1.1_regular,F2.1_regular,F6_reverse,F4_regular,F3_regular,F5.1_regular,F1.2_regular,F2.2_regular,F6.1_regular,F2.3_regular,F4.1_reverse,F2.4_regular,F5.2_reverse,F2.5_regular,F6.2_regular,F1.3_regular,F2.6_regular,F5.3_regular,F4.2_regular,F2.7_regular,F3.1_regular,F2.8_regular,F5.4_regular,F3.2_regular,F1.4_regular,F3.3_regular,F1.5_regular,F5.5_reverse,F6.3_reverse,F1.6_regular,F5.6_reverse,F2.9_regular,F3.4_regular,F4.3_reverse,F2.10_regular,F1.7_regular,F6.4_reverse,F4.4_regular,F5.7_regular,F3.5_regular,F2.11_regular
i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
1,"""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree"""
2,"""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Neither Agree nor Disagree""","""Neither Agree nor Disagree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree"""
3,"""Strongly Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Strongly Agree""","""Strongly Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Strongly Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Disagree""","""Strongly Agree""","""Somewhat Agree""","""Strongly Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""",null,"""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Strongly Agree""","""Somewhat Disagree""","""Somewhat Disagree""","""Strongly Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Strongly Agree""","""Strongly Disagree""","""Somewhat Agree"""
4,"""Somewhat Agree""","""Somewhat Agree""","""Strongly Agree""","""Somewhat Agree""","""Strongly Agree""","""Neither Agree nor Disagree""","""Neither Agree nor Disagree""","""Somewhat Disagree""","""Somewhat

In [217]:
import polars.selectors as cs

In [218]:
#3
mapping = {
    "Strongly Disagree": 1,
    "Somewhat Disagree": 2,
    "Neither Agree nor Disagree": 3,
    "Somewhat Agree": 4,
    "Strongly Agree": 5
}
#gave default value-3 so that null values are not created and questions ares skewed
#multiplied by -1 and added 6 to reverse the scale for reverse coded questions
healthcare_right = healthcare.with_columns(
    cs.contains("_regular").replace(mapping).cast(int).fill_null(3),      #step 5/1 to get rid of null values
    (cs.contains("_reverse").replace(mapping).cast(int) * -1 + 6).fill_null(3)
)
healthcare_right.head()



ID,F1_regular,F5_reverse,F2_regular,F1.1_regular,F2.1_regular,F6_reverse,F4_regular,F3_regular,F5.1_regular,F1.2_regular,F2.2_regular,F6.1_regular,F2.3_regular,F4.1_reverse,F2.4_regular,F5.2_reverse,F2.5_regular,F6.2_regular,F1.3_regular,F2.6_regular,F5.3_regular,F4.2_regular,F2.7_regular,F3.1_regular,F2.8_regular,F5.4_regular,F3.2_regular,F1.4_regular,F3.3_regular,F1.5_regular,F5.5_reverse,F6.3_reverse,F1.6_regular,F5.6_reverse,F2.9_regular,F3.4_regular,F4.3_reverse,F2.10_regular,F1.7_regular,F6.4_reverse,F4.4_regular,F5.7_regular,F3.5_regular,F2.11_regular
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
1,4,4,4,4,4,4,4,4,4,4,4,4,4,2,4,2,4,4,4,4,4,4,4,2,4,4,4,3,4,4,4,4,4,2,4,2,3,4,4,2,4,4,4,4
2,4,4,4,4,4,4,4,3,3,4,3,4,4,3,4,2,4,4,4,4,4,4,4,3,4,3,3,4,3,3,3,4,4,4,4,4,3,4,4,4,3,4,3,4
3,5,3,4,5,5,4,4,4,4,5,4,3,4,3,4,4,5,4,5,4,4,4,3,2,4,4,4,2,4,5,4,4,5,4,4,4,3,4,4,2,4,5,1,4
4,4,2,5,4,5,3,3,2,4,5,5,4,4,3,5,4,5,4,4,5,4,4,4,2,4,4,2,3,2,4,4,2,5,4,4,2,2,4,3,3,3,4,2,4
5,5,5,3,5,4,5,5,4,3,5,4,4,2,3,4,5,5,5,5,5,5,4,4,4,4,3,4,4,4,4,5,3,5,5,4,4,3,4,4,2,4,5,2,4


In [235]:
#4using pl.reduce for each question type
import polars.selectors as cs

healthcare_numberss = healthcare_right.with_columns([
    pl.col(c).alias(c.replace("_regular", "_score").replace("_reverse", "_score"))
    for c in healthcare_right.columns
    if "_regular" in c or "_reverse" in c
]).select("ID", cs.contains("_score")).with_columns([
   pl.reduce(add, cs.contains("F1")).alias("F1_total"),
   pl.reduce(add, cs.contains("F2")).alias("F2_total"),
   pl.reduce(add, cs.contains("F3")).alias("F3_total"),
   pl.reduce(add, cs.contains("F4")).alias("F4_total"),
   pl.reduce(add, cs.contains("F5")).alias("F5_total"),
   pl.reduce(add, cs.contains("F6")).alias("F6_total")
    ]).select("ID", cs.contains("_total"))  

healthcare_numberss.head()

ID,F1_total,F2_total,F3_total,F4_total,F5_total,F6_total
i64,i64,i64,i64,i64,i64,i64
1,31,48,20,17,28,18
2,31,47,19,17,27,20
3,36,49,19,18,32,17
4,32,54,12,15,30,16
5,37,47,22,19,36,19


In [239]:
#5/2using one expression using list comprehension
question_prefixes = ["F1", "F2", "F3", "F4", "F5", "F6"]

healthcareshort = (
    healthcare_right
    .with_columns([
        pl.col(c).alias(c.replace("_regular", "_score").replace("_reverse", "_score"))
        for c in healthcare_right.columns
        if "_regular" in c or "_reverse" in c
    ])
    .select("ID", cs.contains("_score"))
    .with_columns([
        pl.reduce(add, cs.contains(q)).alias(f"{q}_total")
        for q in question_prefixes
    ])
    .select("ID", cs.contains("_total"))
)

healthcareshort.head()


ID,F1_total,F2_total,F3_total,F4_total,F5_total,F6_total
i64,i64,i64,i64,i64,i64,i64
1,31,48,20,17,28,18
2,31,47,19,17,27,20
3,36,49,19,18,32,17
4,32,54,12,15,30,16
5,37,47,22,19,36,19
